In [66]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np 
#for importing files
import glob, os
import matplotlib.pyplot as plt
%matplotlib inline


#Library for hull calculation
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from matplotlib.path import Path
from scipy.spatial import distance


## Helper Methods

In [67]:
def SecsToGameTime(time):
    mins=(int(time)/60)
    secs=int(time)%60
    print (str(mins)+":"+str(secs))

In [68]:
def GameTimetoSeconds(time):
    mins, secs = time.split(":")
    timeSecs = int(mins)*60+int(secs)
    print(timeSecs)
    return timeSecs

#### Hull Calculation Helper Method

In [69]:
def getHull(moments, teamID):
    points = []
    
    if(int(moments[5][1][0]) == teamID):
        for positions in moments[5][1:6]:
            #print positions
            points.append([positions[2],positions[3]])
        print(points)
    else:
        for positions in moments[5][6:11]:
            #print positions
            points.append([positions[2],positions[3]])
        print(points)
    
    np_points = np.asarray(points)
    np_points
    hull = ConvexHull(np_points)
    
    return hull,np_points

In [70]:
def distFromCentroid(hull, x,y):
        cx = np.mean(hull.points[hull.vertices,0])
        cy = np.mean(hull.points[hull.vertices,1])
        return ((x-cx)^2+(y-cy)^2)^.5

In [71]:
def isInHull(hull,np_points,x,y):
    #point in hull?
    hull_path = Path( np_points[hull.vertices] )
    isInPath = hull_path.contains_point((x,y))
    return (isInPath == True)

### Helper method for returning PBP descriptions given game and eventNum

In [123]:
def getPBPInfo(gameID,PBPEventNum):
    events_df = pd.read_csv("00"+str(gameID)+".csv")
    eventRow = events_df[events_df['EVENTNUM'].isin([int(PBPEventNum)])]
    print eventRow['PCTIMESTRING']
    print eventRow['HOMEDESCRIPTION']
    print eventRow['VISITORDESCRIPTION']
    

### Given the game time (12:00) and eventNumber from the Play-by-play data for a recorded event, this method returns the correct eventNumber from the SportsVU data since they actually dont matchup.  This method converts the gametime into seconds (720s = 12:00) which is formatted in SportsVU.  It traverses up and down events until the gametime is within a selected eventNumber.

In [106]:
def findCorrectMomentData(rawSportVUData, quarterPBP, gametimePBP, approxEventNumPBP):
    print("Finding correct SportVU eventNum for gameid:"+rawSportVUData['gameid'])

    found = False;
    moments = None
    #convert gametimePBP (12:00) to seconds (720)
    gameInSeconds = GameTimetoSeconds(gametimePBP)
    while (found == False):
        print("Currently checking eventNumPBP: "+str(approxEventNumPBP))
        moments = MomentsfromEventNum(rawSportVUData, approxEventNumPBP)
        if (moments is None or len(moments) == 0):
            return None
        
        quarterSportVU = int(moments[0][0])
        #print("PBP quareter:"+str(quarterPBP)+"quartersportVU :"+str(quarterSportVU))
        #print("PBP gameclock:"+str(gameInSeconds)+" SportsVU moment clock:"+str(moments[0][2]))
        
        #check whether this EventNum moment is the correct time frame of the play from Play-by-play
        if(quarterPBP < quarterSportVU):
            approxEventNumPBP -= 1
        elif(quarterPBP > quarterSportVU):    
            approxEventNumPBP  += 1
        elif(int(moments[0][2]) > gameInSeconds and int(moments[len(moments)-1][2]) < gameInSeconds):
            found=True
        elif(moments[0][2] < gameInSeconds):
            approxEventNumPBP -= 1
        elif(moments[len(moments)-1][2] > gameInSeconds):
        # look ahead 1 event 
            approxEventNumPBP += 1
        else:
            return None
    #print("extracted event num "+ str(approxEventNumPBP))
    return approxEventNumPBP

In [117]:
def MomentsfromEventNum(eventsData, eventNumPBP):
    found = False
    searchDirection = None
    sportVUIndex = eventNumPBP
    eventsLen = len(eventsData['events'])-1
    if(eventsLen < sportVUIndex):
        sportVUIndex = eventsLen
    
    while(not found):        
        if( int(eventsData['events'][sportVUIndex]['eventId']) == eventNumPBP):
            found = True
        elif ( int(eventsData['events'][sportVUIndex]['eventId']) > eventNumPBP ):
            if(searchDirection is None and sportVUIndex > 0):
                searchDirection = "decrementIndex"
                sportVUIndex -= 1  
            elif(searchDirection is "decrementIndex" and sportVUIndex > 0):
                sportVUIndex -= 1  
            elif(searchDirection is "incrementIndex"):
                return None
            else: #we have reached 0th index and cant traverse up the list anymore
                return None
        elif ( int(eventsData['events'][sportVUIndex]['eventId']) < eventNumPBP ):
            if(searchDirection is None and sportVUIndex < eventsLen):
                searchDirection = "incrementIndex"
                sportVUIndex += 1  
            elif(searchDirection is "incrementIndex" and sportVUIndex < eventsLen):
                sportVUIndex += 1  
            elif(searchDirection is "decrementIndex"):
                return None
            else: #we have reached 0th index and cant traverse up the list anymore
                return None
    print("sportVU Index for the eventNumPBP: "+str(sportVUIndex))
        
    return eventsData['events'][sportVUIndex]["moments"]

## Identify Moment of CATCH

1.  //sort events moment data from end time to beginning (reverse order)
timeballReceived, locationballReceived = getReceive (recieverID, List[Moments]) 

Each moment is taken every .25s, so the FIRST 8 moments (8 *.25 = 2 seconds) when the receiverID and ball are within (dist <= 1 ft), is when the ball was the moment before the ball was released.

reset numCount = 0;

Starting from this time, continue iterating through moments. If dist from receiver and ball >=1 ft, then this is time the ball was caught.

***separate calculation to check when the ball dist less than 1 ft for another teammate to determine if it was a pass in a certain amount of time (counted numCount)****

2.  Find "latest" instance when   distFrom ( Moment.Positions[0].x,Moment.Position[0].y , Moment.Position[ ??=playerID ].x , Moment.Position[ ??=playerID].y) <= 1ft )

   return the timemilis ,  position[0].x , position[0].y


In [103]:
 def getCatchEventDetails(receiverID, passerID, gametimePBP, moments):
    momentsReversed = moments[::-1]
    processComplete = False;
    indexBallCaught = None
    indexBallReleased = None
    index=0
    found = False
    gameInSeconds = GameTimetoSeconds(gametimePBP)
    print("num of moments: "+str(len(momentsReversed)))
    while (found == False):
        if(int(momentsReversed[index][2]) >=  gameInSeconds):
            found=True
        elif(int(momentsReversed[index][2]< gameInSeconds)):
            index += 1
        else:
            return "ERROR"
    
    print("starting at time: "+ str(momentsReversed[index][2]))
    
    #Found the index of the approx start of the event
    numCounts=0
    for i in range(index,len(momentsReversed)):
        if numCounts >= 3:
            timeBallReleased = momentsReversed[i][2]
            SecsToGameTime(timeBallReleased)  
            indexBallReleased = i
            break
        else:
            ball = momentsReversed[i][5][0]
            for positions in momentsReversed[i][5]:
                if positions[1] == receiverID:
                    player_x = positions[2]
                    player_y = positions[3]
            dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
            if dist <= 2:
                print("receiver holding ball: " + str( momentsReversed[i][3]))
                numCounts += 1
    
    #check if ball caught event in moments list
    if(indexBallReleased is None):            
        return None, None, None, None, None, None, None, None, None  
    
    #count when ball is caught
    numCounts=0
    for j in range(indexBallReleased, len(momentsReversed)):
        if numCounts >= 3:
            timeBallCaught = momentsReversed[j][2]
            SecsToGameTime(timeBallCaught)
            indexBallCaught = j
            
            #record position of receiver
            receiverMoment = momentsReversed[j][5]
            receiverPos = [k for k in receiverMoment if receiverID in k]
            
            reciever_pos_x = receiverPos[0][2]
            receiver_pos_y = receiverPos[0][3]
            receiverPosAtCaught = [reciever_pos_x, receiver_pos_y]
            
            break
        else:
            ball = momentsReversed[j][5][0]
            for positions in momentsReversed[j][5]:
                if positions[1] == receiverID:
                    player_x = positions[2]
                    player_y = positions[3]
            dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
            if dist >= 2:
                print("receiver just caught ball: " + str(momentsReversed[j][3])+" received reversed index"+str(j))
                SecsToGameTime(momentsReversed[j][3])
                numCounts += 1
    
    #check if ball caught event in moments list
    if(indexBallCaught is None):            
        return None, None, None, None, None, None, None, None, None            
                
                
    #count when ball is passed
    numCounts=0
    for k in range(indexBallCaught, len(momentsReversed)):
        if numCounts >= 2:
            print("player PASSED ball at "+str(momentsReversed[k][3]))
            timeBallPassed = momentsReversed[k][2]
            SecsToGameTime(timeBallPassed)
            indexBallPassed = k
            processComplete = True
               
            #record position of receiver
            receiverMoment = momentsReversed[k][5]
            receiverPos = [k for k in receiverMoment if receiverID in k]
            
            reciever_pos_x = receiverPos[0][2]
            receiver_pos_y = receiverPos[0][3]
            receiverPosAtPass = [reciever_pos_x ,receiver_pos_y]
        
            #record position of passer
            passerPos = [k for k in receiverMoment if passerID in k]
            
            passer_pos_x = passerPos[0][2]
            passer_pos_y = passerPos[0][3]
            passerPosAtPass = [passer_pos_x ,passer_pos_y]
            
            
            break
        else:
            ball = momentsReversed[k][5][0]
            for positions in momentsReversed[k][5]:
                if positions[1] == passerID:
                    player_x = positions[2]
                    player_y = positions[3]
            dist = distance.euclidean([player_x,player_y],[ball[2],ball[3]])
            if dist <= 2:
                print("passer just passed ball: " +str(momentsReversed[k][3]) + " passed reversed index"+str(k))
                numCounts += 1
                
    
    if(not processComplete):            
        return None, None, None, None, None, None, None, None, None
    
        
    return timeBallReleased, indexBallReleased, timeBallCaught, indexBallCaught, timeBallPassed, indexBallPassed, receiverPosAtPass, receiverPosAtCaught, passerPosAtPass 
   

### Extracts Assists, Bad Passes, Alley Oops from csvs in current directory and saves them into separate csv files

In [75]:
AST_df = pd.DataFrame()
BadPass_df = pd.DataFrame()
AlleyOop_df = pd.DataFrame()
curPath = os.getcwd()
print curPath
os.chdir(curPath)
for pbpFile in glob.glob("*.csv"):
    print(pbpFile)
    #read file as df
    events_df = pd.read_csv(pbpFile)
    ASTList = events_df[events_df['EVENTMSGTYPE'].isin([1,2]) & ( events_df['VISITORDESCRIPTION'].str.contains("AST", na = False ) | events_df['HOMEDESCRIPTION'].str.contains("AST", na = False )  ) ]
    BadPassList= events_df[( events_df['VISITORDESCRIPTION'].str.contains("Bad", na = False ) | events_df['HOMEDESCRIPTION'].str.contains("Bad", na = False )  ) ]
    AlleyOopList= events_df[events_df['EVENTMSGTYPE'].isin([1,2]) & ( events_df['VISITORDESCRIPTION'].str.contains("Alley", na = False ) | events_df['HOMEDESCRIPTION'].str.contains("Alley", na = False )  ) ]
    AST_df = AST_df.append(ASTList)
    BadPass_df = BadPass_df.append(BadPassList)
    AlleyOop_df = AlleyOop_df.append(AlleyOopList)
    
AST_df.to_csv("AST_Parsed.csv", sep='\t')
BadPass_df.to_csv("BadPass_Parsed.csv", sep='\t')
AlleyOop_df.to_csv("AlleyOop_Parsed.csv", sep='\t')

C:\Users\jchin\Documents\GitHub\Data Mining Course\DataMining-Project-Folder\JupyterNotebooks\pbp-csv
0021500003.csv
0021500004.csv


## AST_df  assists data frame has time and event num of all assists....

In [120]:
AST_df

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION
4,21500003,4,1,1,1,11:03 PM,11:35,Green 26' 3PT Jump Shot (3 PTS) (Thompson 1 AST),NaN,NaN,...,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN
5,21500003,5,1,46,1,11:03 PM,11:18,NaN,NaN,Davis 7' Running Jump Shot (2 PTS) (Gordon 1 AST),...,New Orleans,Pelicans,NOP,0,0,NaN,NaN,NaN,NaN,NaN
6,21500003,6,1,99,1,11:04 PM,10:56,Curry 3' Cutting Finger Roll Layup Shot (2 PTS...,NaN,NaN,...,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN
11,21500003,11,1,58,1,11:04 PM,10:10,NaN,NaN,Perkins 11' Turnaround Hook Shot (2 PTS) (Gord...,...,New Orleans,Pelicans,NOP,0,0,NaN,NaN,NaN,NaN,NaN
20,21500003,20,1,63,1,11:06 PM,9:00,NaN,NaN,Perkins 9' Fadeaway Jumper (4 PTS) (Robinson 1...,...,New Orleans,Pelicans,NOP,0,0,NaN,NaN,NaN,NaN,NaN
33,21500003,32,1,42,1,11:08 PM,7:45,NaN,NaN,Cunningham 3' Driving Layup (2 PTS) (Perkins 1...,...,New Orleans,Pelicans,NOP,0,0,NaN,NaN,NaN,NaN,NaN
34,21500003,34,1,108,1,11:08 PM,7:29,Bogut Cutting Dunk Shot (4 PTS) (Curry 1 AST),NaN,NaN,...,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN
37,21500003,37,1,3,1,11:09 PM,6:56,NaN,NaN,Perkins 6' Hook Shot (6 PTS) (Smith 1 AST),...,New Orleans,Pelicans,NOP,0,0,NaN,NaN,NaN,NaN,NaN
40,21500003,41,1,75,1,11:10 PM,6:22,Thompson 1' Driving Finger Roll Layup (2 PTS) ...,NaN,NaN,...,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN
42,21500003,45,1,58,1,11:13 PM,6:04,NaN,NaN,Perkins 8' Turnaround Hook Shot (8 PTS) (Davis...,...,New Orleans,Pelicans,NOP,0,0,NaN,NaN,NaN,NaN,NaN


In [121]:
# for each assist event ....
print("Starting unparsed pass dataset: "+ str(len(AST_df)))
CleanData_df = pd.DataFrame(columns=['SportVU eventNum', 'time ball passed', 'time ball caught', 'passInHull', 'receiverinHull'])
curPath = os.getcwd()
print curPath
SportVU_JSON_Rootpath = "C:\Users\jchin\Documents\GitHub\Data Mining Course\DataMining-Project-Folder\JupyterNotebooks\SportVU_JSON_Data\\"
#print SportVU_JSON_Rootpath+str(gameid)+".json"
gameid_prev = 0;
for index, ast in AST_df.iterrows():
    gameid = ast["GAME_ID"]    #game id from PBP
    eventNumPBP = ast["EVENTNUM"] #eventNumber for the play
    gametimePBP = ast["PCTIMESTRING"]  #game time for the play
    quarterPBP = int(ast["PERIOD"])
    getPBPInfo(gameid,eventNumPBP)
    print("prevGameID:"+str(gameid_prev)+", currentlyAnalyzingGameID:"+str(gameid))
    print("PBPEventNum:"+str(eventNumPBP)+", gametimePBP:"+str(gametimePBP))
    #check if SportVU json data is already opened for this PBP play
    if(gameid_prev != gameid):
        #open raw JSON SportVU data using game id from assist
        with open(SportVU_JSON_Rootpath+"00"+str(gameid)+".json") as data_file:    
            rawSportVUData = json.load(data_file)
            json_normalize(rawSportVUData['events'])
        gameid_prev = gameid
    
    #get the correct eventNum (returned as long) in SportsVU
    eventNumSportsVU = findCorrectMomentData(rawSportVUData, quarterPBP, gametimePBP, eventNumPBP)
    if(eventNumSportsVU is None):
        print("Could not find eventNum in SportsVU")
        continue
    
    
    print("Correct SportVU eventNum: "+str(eventNumSportsVU))
    #extract SportsVU movements for that eventNum
    
    rawpossessionSportsVU = MomentsfromEventNum(rawSportVUData, int(eventNumSportsVU))
    
    #for analyzing AST only...
    receiverID = int(ast['PLAYER1_ID'])
    passerID = int(ast['PLAYER2_ID'])
    teamID = int(ast['PLAYER1_TEAM_ID'])
                     
    #get catch details (who was the passer, receiver, and when caught/passed)
    timeBallReleased, indexBallReleased, timeBallCaught, indexBallCaught, timeBallPassed, indexBallPassed, receiverPosAtPass, receiverPosAtCaught, passerPosAtPass= getCatchEventDetails(receiverID, passerID, gametimePBP, rawpossessionSportsVU)
    
    if(timeBallReleased is None):
        print("Could not find time for when ball was passed")
        continue
    
    #hull details
    momentsReversed = rawpossessionSportsVU[::-1]
    hull, np_points = getHull(momentsReversed[indexBallCaught], teamID)
                     
    #passer in hull?
    
    passerInHull = isInHull(hull,np_points, passerPosAtPass[0] ,passerPosAtPass[1])
                     
    #receiver in hull?
    receiverInHull = isInHull(hull,np_points, receiverPosAtPass[0] ,receiverPosAtPass[1])
    
    
    
    CleanData_df.loc[len(CleanData_df.index)] = [eventNumSportsVU , timeBallPassed, timeBallCaught, passerInHull, receiverInHull, SportVUMomentIndex ]
    

CleanData_df.to_csv("CleanedData.csv", sep='\t')

Starting unparsed pass dataset: 87
C:\Users\jchin\Documents\GitHub\Data Mining Course\DataMining-Project-Folder\JupyterNotebooks\pbp-csv
4    Green 26' 3PT Jump Shot (3 PTS) (Thompson 1 AST)
Name: HOMEDESCRIPTION, dtype: object
4    NaN
Name: VISITORDESCRIPTION, dtype: object
prevGameID:0, currentlyAnalyzingGameID:21500003
PBPEventNum:4, gametimePBP:11:35
Finding correct SportVU eventNum for gameid:0021500003
695
Currently checking eventNumPBP: 4
sportVU Index for the eventNumPBP: 3
Correct SportVU eventNum: 4
sportVU Index for the eventNumPBP: 3
695
num of moments: 450
starting at time: 695.01
receiver holding ball: 13.34
receiver holding ball: 13.38
receiver holding ball: 13.42
11:37
receiver just caught ball: 13.74 received reversed index108
0:13
receiver just caught ball: 13.78 received reversed index109
0:13
receiver just caught ball: 13.82 received reversed index110
0:13
11:37
passer just passed ball: 14.53 passed reversed index128
passer just passed ball: 14.57 passed reversed i

receiver just caught ball: 15.35 received reversed index120
0:15
receiver just caught ball: 15.39 received reversed index121
0:15
receiver just caught ball: 15.43 received reversed index122
0:15
4:21
passer just passed ball: 17.08 passed reversed index162
passer just passed ball: 17.12 passed reversed index163
player PASSED ball at 17.16
4:23
[[7.52468, 47.40975], [22.19303, 46.35965], [23.11076, 4.32426], [23.62575, 22.4907], [6.20899, 3.11072]]
62    Curry 26' 3PT Step Back Jump Shot (20 PTS) (Gr...
Name: HOMEDESCRIPTION, dtype: object
62    NaN
Name: VISITORDESCRIPTION, dtype: object
prevGameID:21500003, currentlyAnalyzingGameID:21500003
PBPEventNum:68, gametimePBP:3:57
Finding correct SportVU eventNum for gameid:0021500003
237
Currently checking eventNumPBP: 68
sportVU Index for the eventNumPBP: 61
Correct SportVU eventNum: 68
sportVU Index for the eventNumPBP: 61
237
num of moments: 1001
starting at time: 237.0
receiver holding ball: 16.01
receiver holding ball: 16.04
receiver hol

251    Thompson 13' Step Back Jump Shot (6 PTS) (Bogu...
Name: HOMEDESCRIPTION, dtype: object
251    NaN
Name: VISITORDESCRIPTION, dtype: object
prevGameID:21500003, currentlyAnalyzingGameID:21500003
PBPEventNum:283, gametimePBP:11:43
Finding correct SportVU eventNum for gameid:0021500003
703
Currently checking eventNumPBP: 283
sportVU Index for the eventNumPBP: 251
Currently checking eventNumPBP: 284
sportVU Index for the eventNumPBP: 252
Correct SportVU eventNum: 284
sportVU Index for the eventNumPBP: 252
703
num of moments: 675
starting at time: 703.01
receiver holding ball: 9.67
receiver holding ball: 9.71
receiver holding ball: 9.75
11:46
receiver just caught ball: 10.52 received reversed index620
0:10
receiver just caught ball: 10.57 received reversed index621
0:10
receiver just caught ball: 10.61 received reversed index622
0:10
11:47
passer just passed ball: 11.12 passed reversed index634
passer just passed ball: 11.17 passed reversed index635
player PASSED ball at 11.21
11:47
[

Name: HOMEDESCRIPTION, dtype: object
395    NaN
Name: VISITORDESCRIPTION, dtype: object
prevGameID:21500003, currentlyAnalyzingGameID:21500003
PBPEventNum:460, gametimePBP:11:03
Finding correct SportVU eventNum for gameid:0021500003
663
Currently checking eventNumPBP: 460
sportVU Index for the eventNumPBP: 397
Correct SportVU eventNum: 460
sportVU Index for the eventNumPBP: 397
663
num of moments: 775
starting at time: 663.01
receiver holding ball: 18.36
receiver holding ball: 18.41
receiver holding ball: 18.45
11:4
receiver just caught ball: 18.85 received reversed index65
0:18
receiver just caught ball: 18.9 received reversed index66
0:18
receiver just caught ball: 18.94 received reversed index67
0:18
11:4
passer just passed ball: 20.38 passed reversed index99
passer just passed ball: 20.42 passed reversed index100
player PASSED ball at 20.47
11:5
[[88.0626, 26.80487], [76.71035, 7.24557], [27.88159, 24.22133], [96.9046, 18.35756], [84.18195, 25.31102]]
396    NaN
Name: HOMEDESCRIPTI

Finding correct SportVU eventNum for gameid:0021500004
658
Currently checking eventNumPBP: 7
sportVU Index for the eventNumPBP: 5
Correct SportVU eventNum: 7
sportVU Index for the eventNumPBP: 5
658
num of moments: 466
starting at time: 658.02
receiver holding ball: 10.06
receiver holding ball: 10.1
receiver holding ball: 10.14
11:0
receiver just caught ball: 10.77 received reversed index460
0:10
receiver just caught ball: 10.81 received reversed index461
0:10
receiver just caught ball: 10.89 received reversed index463
0:10
11:1
Could not find time for when ball was passed
18    NaN
Name: HOMEDESCRIPTION, dtype: object
18    Wall 24' 3PT Jump Shot (3 PTS) (Gortat 1 AST)
Name: VISITORDESCRIPTION, dtype: object
prevGameID:21500004, currentlyAnalyzingGameID:21500004
PBPEventNum:21, gametimePBP:9:36
Finding correct SportVU eventNum for gameid:0021500004
576
Currently checking eventNumPBP: 21
sportVU Index for the eventNumPBP: 18
Correct SportVU eventNum: 21
sportVU Index for the eventNumPB

Name: HOMEDESCRIPTION, dtype: object
120    Neal 17' Jump Shot (2 PTS) (Nene 1 AST)
Name: VISITORDESCRIPTION, dtype: object
prevGameID:21500004, currentlyAnalyzingGameID:21500004
PBPEventNum:142, gametimePBP:11:24
Finding correct SportVU eventNum for gameid:0021500004
684
Currently checking eventNumPBP: 142
sportVU Index for the eventNumPBP: 123
Correct SportVU eventNum: 142
sportVU Index for the eventNumPBP: 123
684
num of moments: 625
starting at time: 684.0
receiver holding ball: 10.61
receiver holding ball: 10.65
receiver holding ball: 10.69
11:26
receiver just caught ball: 11.53 received reversed index587
0:11
receiver just caught ball: 11.93 received reversed index597
0:11
receiver just caught ball: 11.97 received reversed index598
0:11
11:27
passer just passed ball: 12.09 passed reversed index601
passer just passed ball: 12.13 passed reversed index602
player PASSED ball at 12.17
11:27
[[31.26628, 18.54566], [19.34857, 40.65847], [23.37131, 42.8105], [13.01169, 11.34668], [19.015

336    Oladipo  3PT Jump Shot (11 PTS) (Harris 3 AST)
Name: HOMEDESCRIPTION, dtype: object
336    NaN
Name: VISITORDESCRIPTION, dtype: object
prevGameID:21500004, currentlyAnalyzingGameID:21500004
PBPEventNum:391, gametimePBP:3:04
Finding correct SportVU eventNum for gameid:0021500004
184
Currently checking eventNumPBP: 391
sportVU Index for the eventNumPBP: 341
Correct SportVU eventNum: 391
sportVU Index for the eventNumPBP: 341
184
num of moments: 532
starting at time: 184.03
receiver holding ball: 9.62
receiver holding ball: 9.66
receiver holding ball: 9.7
3:6
receiver just caught ball: 10.42 received reversed index443
0:10
receiver just caught ball: 10.46 received reversed index444
0:10
receiver just caught ball: 10.5 received reversed index445
0:10
3:7
passer just passed ball: 10.82 passed reversed index453
passer just passed ball: 10.86 passed reversed index454
player PASSED ball at 10.9
3:7
[[28.41873, 14.97125], [10.3609, 23.7997], [11.98718, 17.88399], [8.32112, 2.98065], [0.7

In [ ]:
#for i, row in events_queried.iterrows():
for i in range(events_queried.shape[0]):
    #extract the row from events dataframe
    extractedRow = events_queried.iloc[i]
    print extractedRow
    #movement_events_df.append(extractedRow,ignore_index=True)
    movement_events_df = movement_events_df.append(pd.DataFrame(extractedRow), ignore_index=True)
    #movement_events_df = pd.concat([movement_events_df, row], axis=1)
    

In [124]:
getPBPInfo(21500003,182)

159    7:50
Name: PCTIMESTRING, dtype: object
159    NaN
Name: HOMEDESCRIPTION, dtype: object
159    Cunningham 21' Jump Shot (7 PTS) (Smith 3 AST)
Name: VISITORDESCRIPTION, dtype: object


87